### Automated Milling Workflow

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pprint import pprint

pprint(sys.path)
cache = sys.path
paths = ['/home/meteor', '/home/meteor/development/odemis/src/', '/home/meteor/development/Pyro4/src/', 
         '/home/meteor/development/odemis/src', '/home/meteor/development/Pyro4/src', 
         '/home/meteor', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', 
         '/home/meteor/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']

for path in paths:
    if path not in sys.path:
        sys.path.insert(0, path)

for path in sys.path:
    if path not in paths:
        sys.path.remove(path)
        print(f"removed: {path}")

from pprint import pprint
pprint(sys.path)


from odemis import model
from odemis.acq.acqmng import acquire

from odemis.acq.stream import LiveStream, SEMStream, FIBStream, StaticSEMStream, FluoStream
import os
from pprint import pprint

import numpy as np
from odemis.acq.millmng import run_automated_milling
from odemis.util import executeAsyncTask
from concurrent import futures

In [ ]:
# connect to hardware

# stage
stage = model.getComponent(role="stage-bare")
print(f"Stage Position: {stage.position.value}")

# setup electron beam, det
electron_beam = model.getComponent(role="e-beam")
electron_det = model.getComponent(role="se-detector")
electron_focus = model.getComponent(role="ebeam-focus")

hwemtvas = set()
hwdetvas = set()

hwemt_vanames = ("beamCurrent", "accelVoltage", "resolution", "dwellTime", "horizontalFoV")
hwdet_vanames = ("brightness", "contrast", "detector_mode", "detector_type")
for vaname in model.getVAs(electron_beam):
    if vaname in hwemt_vanames:
        hwemtvas.add(vaname)
for vaname in model.getVAs(electron_det):
    if vaname in hwdet_vanames:
        hwdetvas.add(vaname)

sem_stream = SEMStream(
    name="SEM",
    detector=electron_det,
    dataflow=electron_det.data,
    emitter=electron_beam,
    focuser=electron_focus,
    hwemtvas=hwemtvas,
    hwdetvas=hwdetvas,
    blanker=None)

# setup ion beam, det
ion_beam = model.getComponent(role="ion-beam")
ion_det = model.getComponent(role="se-detector-ion")
ion_focus = model.getComponent(role="ion-focus")

hwemtvas = set()
hwdetvas = set()
# hwemt_vanames = ("beamCurrent", "accelVoltage", "resolution", "dwellTime", "horizontalFoV")
# hwdet_vanames = ("brightness", "contrast", "detector_mode", "detector_type")
for vaname in model.getVAs(ion_beam):
    if vaname in hwemt_vanames:
        hwemtvas.add(vaname)
for vaname in model.getVAs(ion_det):
    if vaname in hwdet_vanames:
        hwdetvas.add(vaname)

fib_stream = FIBStream(
    name="FIB",
    detector=ion_det,
    dataflow=ion_det.data,
    emitter=ion_beam,
    focuser=ion_focus,
    hwemtvas=hwemtvas,
    hwdetvas=hwdetvas,
)

print(f"SEM: {sem_stream}")
print(f"FIB: {fib_stream}")

# acquisitionStreams for FLM

ccd = model.getComponent(role="ccd")
light = model.getComponent(role="light")
focus = model.getComponent(role="focus")
em_filter = model.getComponent(role="filter")
fm_stream = FluoStream(
    name="FM",
    detector=ccd,
    dataflow=ccd.data,
    emitter=light,
    em_filter=em_filter,
    focuser=focus,
    # opm=self._main_data_model.opm,
    # detvas={"exposureTime"},
)

In [ ]:

sem_stream.emitter.horizontalFoV.value = 80e-6
fib_stream.emitter.horizontalFoV.value = 80e-6

f = acquire([sem_stream, fib_stream])

data, err  = f.result()

sem_image, fib_image  = data

%matplotlib inline
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(sem_image, cmap="gray")
ax[1].imshow(fib_image, cmap="gray")
# ax[2].imshow(fm_image, cmap="gray")
plt.show()


In [ ]:
# f = acquire([fm_stream])
# data, err  = f.result()

# fm_image = data

# %matplotlib inline
# from matplotlib import pyplot as plt
# fig, ax = plt.subplots(1, 1, figsize=(5, 5))
# ax.imshow(fm_image[0], cmap="gray")
# plt.show()


In [ ]:
# for task_name, task in milling_tasks.items():
#     print(f"Task: {task_name}")
#     p = task.generate()
#     pprint(p)

## Automated Milling


In [ ]:
from odemis.acq.milling.feature import create_new_project, create_new_feature
from odemis.acq.milling.tasks import load_milling_tasks, MILLING_ROUGH, MILLING_FINE, MILL_POLISHING
from datetime import datetime 


# format as YYYY-MM-DD_HH-MM-SS
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
PROJECT_NAME = f"milling-project-{timestamp}"

PROJECT_PATH = os.path.join(os.getcwd(), "automated-milling-projects")
os.makedirs(PROJECT_PATH, exist_ok=True)

# load milling tasks
task_list = [MILLING_ROUGH, MILLING_FINE, MILL_POLISHING]
milling_tasks = load_milling_tasks(os.path.join(PROJECT_PATH, "milling_tasks.yaml"), task_list)

# create project
project = create_new_project(PROJECT_PATH, PROJECT_NAME)


# features
# create_new_feature(name="Lamella-1",
#     position={"x": 50e-6, "y": 25e-6, "z": 15e-6, "rx": 0, "rz": 0},
#     milling_tasks=milling_tasks,
#     project=project,
# )

# create_new_feature(name="Lamella-1",
#     position={"x": 33e-6, "y": 100e-6, "z": 66e-6, "rx": 0, "rz": 0},
#     milling_tasks=milling_tasks,
#     project=project,
# )

pprint(project.features)

# steps
# 1. create feature
# 2. select position
# 3. select milling tasks
# 4. select alignment
# ready to mill


# TODO: ref image data -> when to acquire reference image
# TODO: save directory -> project directory / feature directory


In [ ]:
while input("Add another position? (y/n)") == "y":
    feature = create_new_feature(name="Lamella-1",
        position=stage.position.value,
        milling_tasks=milling_tasks,
        project=project,
    )
    name = feature.name.value

    sem_stream.emitter.horizontalFoV.value = 80e-6
    fib_stream.emitter.horizontalFoV.value = 80e-6

    f = acquire([sem_stream, fib_stream])

    data, err  = f.result()

    sem_image, fib_image  = data

    %matplotlib inline
    from matplotlib import pyplot as plt
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(sem_image, cmap="gray")
    ax[1].imshow(fib_image, cmap="gray")
    # ax[2].imshow(fm_image, cmap="gray")
    plt.show()

    project.features[name].reference_image = fib_image
    
    pprint(project.features)

    project.save()

In [ ]:
# ref_images = {}

In [ ]:
# for name, feature in project.features.items():

#     f = stage.moveAbs(feature.position.value)
#     f.result()

#     sem_stream.emitter.horizontalFoV.value = 100e-6
#     fib_stream.emitter.horizontalFoV.value = 100e-6

#     f = acquire([sem_stream, fib_stream])

#     data, err  = f.result()

#     sem_image, fib_image  = data

#     %matplotlib inline
#     from matplotlib import pyplot as plt
#     fig, ax = plt.subplots(1, 2, figsize=(10, 5))
#     ax[0].imshow(sem_image, cmap="gray")
#     ax[1].imshow(fib_image, cmap="gray")
#     # ax[2].imshow(fm_image, cmap="gray")
#     plt.show()

#     feature.reference_image = fib_image



In [ ]:

# f = stage.moveAbs({"x": 0, "y": 0, "z": 0, "rx": 0, "rz": 0})
# f.result()


In [ ]:
import os
from odemis import model
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

from odemis.acq.milling.tasks import draw_milling_tasks

for k, f in project.features.items():

    ref_image = f.reference_image

    fig = draw_milling_tasks(ref_image, milling_tasks)
    plt.show()

In [ ]:
# loop through each position, mill lamella

# TODO: set reference image correctly
# for k, f in project.features.items():
    # f.reference_image = fib_image


f: model.ProgressiveFuture = run_automated_milling(stage=stage, 
                          sem_stream=sem_stream, 
                          fib_stream=fib_stream, 
                          fm_stream=fm_stream, 
                          task_list=task_list, #[MILL_POLISHING], 
                          project=project)

def on_milling_done(f):
    print("Milling Done")

def update_progress(future, start, end):
    if hasattr(future, "msg"):
        print(f"PROGRESS UPDATED {future.msg}, {start}, {end}")

f.add_update_callback(update_progress)
f.add_done_callback(on_milling_done)
f.result()


In [ ]:
from odemis.acq.millmng import mill_patterns

milling_tasks = load_milling_tasks(os.path.join(PROJECT_PATH, "milling_tasks.yaml"), task_list)


sem_stream.emitter.horizontalFoV.value = 80e-6
fib_stream.emitter.horizontalFoV.value = 80e-6

f = acquire([sem_stream, fib_stream])

data, err  = f.result()

sem_image, fib_image  = data

%matplotlib inline
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(sem_image, cmap="gray")
ax[1].imshow(fib_image, cmap="gray")
# ax[2].imshow(fm_image, cmap="gray")
plt.show()

task1 = milling_tasks[MILLING_ROUGH]

print(task1.patterns)

task2 = milling_tasks[MILLING_FINE]

print(task2.patterns)

task3 = milling_tasks[MILL_POLISHING]

print(task3.patterns)

task1.milling.current.value = 41e-12
task2.milling.current.value = 41e-12
task3.milling.current.value = 41e-12

# fig = draw_milling_tasks(fib_image, milling_tasks)
# plt.show()

for task in [task1, task2, task3]:

    f = mill_patterns(task)
    f.result()


# OVERVIEW Acquisitions

In [ ]:
from odemis.acq.stitching import get_tiled_areas, get_zstack_levels, acquireTiledArea, acquireOverview, FocusingMethod, REGISTER_IDENTITY, WEAVER_MEAN
from odemis.util.filename import create_filename
from odemis.dataio import find_fittest_converter

sample_centers_raw = stage.getMetadata()[model.MD_SAMPLE_CENTERS]
sample_centers = {n: (p["x"], p["y"]) for n, p in sample_centers_raw.items()}

# move to base
# stage.moveAbs({"x": 0, "y": 0, "z": 0, "rx": 0, "rz": 0}).result()

electron_beam.horizontalFoV.value = 500e-6
acq_streams = [sem_stream]

print(sample_centers)
areas = get_tiled_areas(
    pos=stage.position.value,
    streams=acq_streams,
    tiles_nx=2,
    tiles_ny=2,
    overlap=0.1,
    tiling_rng={"x": [-1e-3, 1e-3], "y": [-1e-3, 1e-3]},
    selected_grids=["GRID 1", "GRID 2"],
    sample_centers=sample_centers,
    whole_grid=True,
)

print(areas)

# BLOCKED WHILE THE UI IS OPEN????

In [ ]:
import matplotlib.pyplot as plt
for da in data:

    plt.imshow(da, cmap="gray")
    plt.show()

In [ ]:
data = acq_future.result(1)  # timeout is just for safety

In [ ]:
pprint(data[0].metadata)

In [ ]:
fib_stream.focuser.axes["z"].range

In [ ]:
from odemis.util.dataio import open_acquisition
import matplotlib.pyplot as plt

image = open_acquisition("/home/patrick/development/scratch/acq/grids/20240611-183438-overview.ome.tiff")

plt.imshow(image[0], cmap="gray")
plt.show()

In [ ]:
from odemis import model

stage = model.getComponent(role="stage-bare")
stage.moveAbs({"x": 0, "y": 0, "z": 0, "rx": 0, "rz": 0}).result()

In [ ]:
# SAMPLE CENTERS is posture dependent
# therefore it should be converted to a posture dependent value

print(sample_centers)

In [ ]:
from odemis import model

stage = model.getComponent(role="stage-bare")
sample_centers_raw = stage.getMetadata()[model.MD_SAMPLE_CENTERS]
sample_centers = {n: (p["x"], p["y"]) for n, p in sample_centers_raw.items()}

# get sem orientation
stage_md = stage.getMetadata()
sem_pos_active = stage_md[model.MD_FAV_SEM_POS_ACTIVE]
fm_pos_active = stage_md[model.MD_FAV_FM_POS_ACTIVE]


# update each sample center

for pos in sample_centers_raw.values():
    pos.update(sem_pos_active)
    print(pos)

print("sem: ", sem_pos_active)
print("fm: ", fm_pos_active)
print(sample_centers_raw)

from odemis.acq.move import MicroscopePostureManager, MeteorTFS1PostureManager, SEM_IMAGING, FM_IMAGING, POSITION_NAMES

pm: MeteorTFS1PostureManager = MicroscopePostureManager(model.getMicroscope())

posture = POSITION_NAMES[pm.getCurrentPostureLabel(pos)]
print(posture)

fm_pos_grid_01 = pm._transformFromSEMToMeteor(sample_centers_raw["GRID 1"])
fm_pos_grid_02 = pm._transformFromSEMToMeteor(sample_centers_raw["GRID 2"])

print("FM Positions")
print(fm_pos_grid_01)
print(fm_pos_grid_02)


grid_centres = {
    SEM_IMAGING: {
        "POSITIONS": {
        "GRID 1": sample_centers_raw["GRID 1"],
        "GRID 2": sample_centers_raw["GRID 2"],
    },
    "RANGE": stage_md[model.MD_SEM_IMAGING_RANGE],
    "STREAM": sem_stream,
    "FOCUSER": electron_focus,
    "DETECTOR": electron_det,
},
FM_IMAGING: {
    "POSITIONS": {
        "GRID 1": fm_pos_grid_01,
        "GRID 2": fm_pos_grid_02,
    },
    "RANGE": stage_md[model.MD_FM_IMAGING_RANGE],
    "STREAM": fm_stream,
    "FOCUSER": focus,
    "DETECTOR": ccd,
}
}


In [ ]:



stage = model.getComponent(role="stage-bare")



grid = "GRID 1"
posture = FM_IMAGING
grid_pos = grid_centres[posture]["POSITIONS"][grid]

stage.moveAbs(grid_pos).result() # use pm for safety

grid_positions = grid_centres[posture]["POSITIONS"].items()
grid_centre_tuple = {n: (p["x"], p["y"]) for n, p in grid_positions}
acq_streams = [grid_centres[posture]["STREAM"]]
tiling_rng =  grid_centres[posture]["RANGE"]

focuser = grid_centres[posture]["FOCUSER"]
detector = grid_centres[posture]["DETECTOR"]


# print details:
print(f"Grid: {grid}")
print(f"Posture: {POSITION_NAMES[posture]}")
print(f"Grid Position: {grid_pos}")
print(f"Grid Centre Tuple: {grid_centre_tuple}")
print(f"Streams: {acq_streams}")
print(f"Tiling Range: {tiling_rng}")
print(f"Focuser: {focuser}")
print(f"Detector: {detector}")

stage = model.getComponent(role="stage")
grid_pos = stage.position.value

areas = get_tiled_areas(
    pos=grid_pos,
    streams=acq_streams,
    tiles_nx=2,
    tiles_ny=2,
    overlap=0.1,
    tiling_rng=stage.range,
    selected_grids=["GRID 1"], #, "GRID 2"],
    sample_centers=grid_centre_tuple,
    whole_grid=False,
)

print(areas)

# TODO: these areas need to be projected along the sample plane

# overview settings
focus_mtd = FocusingMethod.NONE
use_autofocus = False
overlap = 0.1

save_dir = os.path.join("/home/patrick/development/scratch", f"acq/grid-fm-frame")
os.makedirs(save_dir, exist_ok=True)
filename = create_filename(save_dir, "{datelng}-{timelng}-overview",
                                        ".ome.tiff")
print(f"Filename: {filename}")
assert filename.endswith(".ome.tiff")
dirname, basename = os.path.split(filename)
tiles_dir = os.path.join(dirname, basename[:-len(".ome.tiff")] + "-tiles")
filename_tiles = os.path.join(tiles_dir, basename)
os.makedirs(tiles_dir, exist_ok=True)

print(filename_tiles)
print(f"Acquiring ...")


stage_fm = model.getComponent(role="stage")
acq_future = acquireOverview(
                streams=acq_streams,              # restrict to a single stream if using autofocus (sem, fib)
                stage=stage_fm, 
                areas=areas, 
                focus=focuser,             # replace with sem-focus, fib-focus
                detector=focuser,            # replace with detector
                overlap=overlap, 
                settings_obs=None, 
                log_path=filename_tiles, 
                zlevels=None,
                registrar=REGISTER_IDENTITY, 
                weaver=WEAVER_MEAN, 
                focusing_method=FocusingMethod.NONE,
                use_autofocus=use_autofocus)

data = acq_future.result()

exporter = find_fittest_converter(filename)
exporter.export(filename, data)


In [ ]:
import matplotlib.pyplot as plt
for da in data:

    plt.imshow(da, cmap="gray")
    plt.show()

In [ ]:
da.metadata

#### HW TEST OVERVIEW

In [ ]:
from odemis.acq.stitching import get_tiled_areas, get_zstack_levels, acquireTiledArea, acquireOverview, FocusingMethod, REGISTER_IDENTITY, WEAVER_MEAN
from odemis.util.filename import create_filename
from odemis.dataio import find_fittest_converter


linked_yz_stage = model.getComponent(name="Linked YZ")
linked_yz_stage.updateMetadata({model.MD_ROTATION_COR: 0.663225})
print(linked_yz_stage.getMetadata())
# 0.663225


stage = model.getComponent(role="stage")
stage_md = stage.getMetadata()
tiling_rng = {
    "x": stage.axes["x"].range,
    "y": stage.axes["y"].range
}

focuser = electron_focus
det = electron_det

electron_beam.horizontalFoV.value = 200e-6


grid_pos = stage.position.value
acq_streams = [sem_stream]

areas = get_tiled_areas(
    pos=grid_pos,
    streams=acq_streams,
    tiles_nx=2,
    tiles_ny=2,
    overlap=0.1,
    tiling_rng=tiling_rng
)

print([f"{a:2e}" for a in areas[0]])


In [ ]:

# TODO: these areas need to be projected along the sample plane

# overview settings
focus_mtd = FocusingMethod.NONE
use_autofocus = False
overlap = 0.1

save_dir = os.path.join(os.getcwd(), f"acq/grid-fm-frame")
os.makedirs(save_dir, exist_ok=True)
filename = create_filename(save_dir, "{datelng}-{timelng}-overview",
                                        ".ome.tiff")
print(f"Filename: {filename}")
assert filename.endswith(".ome.tiff")
dirname, basename = os.path.split(filename)
tiles_dir = os.path.join(dirname, basename[:-len(".ome.tiff")] + "-tiles")
filename_tiles = os.path.join(tiles_dir, basename)
os.makedirs(tiles_dir, exist_ok=True)
input()
# stage_fm = model.getComponent(role="stage")
acq_future = acquireOverview(
                streams=acq_streams,              # restrict to a single stream if using autofocus (sem, fib)
                stage=stage, 
                areas=areas, 
                focus=focuser,             # replace with sem-focus, fib-focus
                detector=det,            # replace with detector
                overlap=overlap, 
                settings_obs=None, 
                log_path=filename_tiles, 
                zlevels=None,
                registrar=REGISTER_IDENTITY, 
                weaver=WEAVER_MEAN, 
                focusing_method=FocusingMethod.NONE,
                use_autofocus=use_autofocus)

data = acq_future.result()

exporter = find_fittest_converter(filename)
exporter.export(filename, data)

import matplotlib.pyplot as plt

plt.imshow(data[0], cmap="gray")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(data[0], cmap="gray")
plt.show()

## 3DCT Importer

In [ ]:
import os
import glob
import logging

from odemis import model
from odemis.util.dataio import open_acquisition
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt


%matplotlib inline

# PATH = "/home/patrick/development/data/CORRELATION/fib-flm/HeLa_cells_01"
PATH = "/home/patrick/development/data/CORRELATION/3dct"

txt_file = glob.glob(os.path.join(PATH, "*.txt"))[0]

filename = os.path.join(PATH, "FOV4_IB_before.tif")
# filename = glob.glob(os.path.join(PATH, "FIB_images", "*before.tif"))[0]
target_positions = []


def parse_3dct_txt_file(filename):
    target_positions = []

    parse = False
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "Distance in px" in line:
                parse = True
            if parse:
                try:
                    line = line.strip()
                    items = line.split("\t")
                    # drop index 2 (empty)
                    items = items[:2] + items[3:]
                    items = [float(i.strip()) for i in items]
                    dpx, dpy = items[0], items[1]
                    dmx, dmy = items[2] * 1e-6, items[3] * 1e-6
                    target_positions.append({"px": dpx, "py": dpy, "mx": dmx, "my": dmy})
                except Exception as e:
                    logging.debug(e)
    return target_positions

target_positions = parse_3dct_txt_file(txt_file)    

# open image
image = open_acquisition(filename)[0]
image = image.getData()

points_px, points_m = [], []
pixel_size = image.metadata[model.MD_PIXEL_SIZE][0]

# parse each target position, convert to pixels and plot
for pos in target_positions:
    
    # position in metres from image centre
    mx, my = float(pos["mx"]), float(pos["my"])
    # convert to pixels
    pmx, pmy = mx / image.metadata[model.MD_PIXEL_SIZE][0], my / image.metadata[model.MD_PIXEL_SIZE][1]
    
    # convert to image coordinates
    cy, cx = image.shape[0] // 2, image.shape[1] // 2
    px = cx + pmx
    py = cy - pmy
    
    # store points
    points_px.append((px, py))
    points_m.append((mx, my))

# get centroid
px, py = np.mean([c[0] for c in points_px]), np.mean([c[1] for c in points_px])

fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(image, cmap="gray")

for i, (px, py) in enumerate(points_px):
    plt.plot(px, py, "r+", ms=5)
    # plt.text(px, py, f"{i}", fontsize=8, color="white")

# select position
rand_idx = np.random.randint(0, len(points_px))
px, py = points_px[rand_idx]
mx, my = points_m[rand_idx]
plt.plot(px, py, "c+", ms=10)
plt.plot(cx, cy, "y+", ms=20)


from odemis.acq.milling.tasks import load_milling_tasks, draw_milling_tasks

PROJECT_PATH = "/home/patrick/development/scratch/project/automated-milling/"

# load milling tasks
milling_tasks = load_milling_tasks(os.path.join(PROJECT_PATH, "milling_tasks.yaml"))

# set milling task centres
for task_name, task in milling_tasks.items():

    for p in task.patterns:
        p.center.value = (mx, my)


fig = draw_milling_tasks(image, milling_tasks)
plt.show()

# TODO: use this code to set the milling task centres

# New Automated Milling Manager (+Integrated Features)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
from pprint import pprint

# pprint(sys.path)
# cache = sys.path
# paths = ['/home/meteor', '/home/meteor/development/odemis/src/', '/home/meteor/development/Pyro4/src/', 
#          '/home/meteor/development/odemis/src', '/home/meteor/development/Pyro4/src', 
#          '/home/meteor', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', 
#          '/home/meteor/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']

# for path in paths:
#     if path not in sys.path:
#         sys.path.insert(0, path)

# for path in sys.path:
#     if path not in paths:
#         sys.path.remove(path)
#         print(f"removed: {path}")

# from pprint import pprint
# pprint(sys.path)


from odemis import model
from odemis.acq.acqmng import acquire

from odemis.acq.stream import LiveStream, SEMStream, FIBStream, StaticSEMStream, FluoStream
import os
from pprint import pprint

import numpy as np
import logging 
import matplotlib.pyplot as plt
from odemis.acq.millmng import run_automated_milling

logging.basicConfig(level=logging.DEBUG)

In [ ]:
# connect to hardware

# stage
stage = model.getComponent(role="stage-bare")
print(f"Stage Position: {stage.position.value}")

# setup electron beam, det
electron_beam = model.getComponent(role="e-beam")
electron_det = model.getComponent(role="se-detector")
electron_focus = model.getComponent(role="ebeam-focus")

hwemtvas = set()
hwdetvas = set()

hwemt_vanames = ("beamCurrent", "accelVoltage", "resolution", "dwellTime", "horizontalFoV")
hwdet_vanames = ("brightness", "contrast", "detector_mode", "detector_type")
for vaname in model.getVAs(electron_beam):
    if vaname in hwemt_vanames:
        hwemtvas.add(vaname)
for vaname in model.getVAs(electron_det):
    if vaname in hwdet_vanames:
        hwdetvas.add(vaname)

sem_stream = SEMStream(
    name="SEM",
    detector=electron_det,
    dataflow=electron_det.data,
    emitter=electron_beam,
    focuser=electron_focus,
    hwemtvas=hwemtvas,
    hwdetvas=hwdetvas,
    blanker=None)

# setup ion beam, det
ion_beam = model.getComponent(role="ion-beam")
ion_det = model.getComponent(role="se-detector-ion")
ion_focus = model.getComponent(role="ion-focus")

hwemtvas = set()
hwdetvas = set()
# hwemt_vanames = ("beamCurrent", "accelVoltage", "resolution", "dwellTime", "horizontalFoV")
# hwdet_vanames = ("brightness", "contrast", "detector_mode", "detector_type")
for vaname in model.getVAs(ion_beam):
    if vaname in hwemt_vanames:
        hwemtvas.add(vaname)
for vaname in model.getVAs(ion_det):
    if vaname in hwdet_vanames:
        hwdetvas.add(vaname)

fib_stream = FIBStream(
    name="FIB",
    detector=ion_det,
    dataflow=ion_det.data,
    emitter=ion_beam,
    focuser=ion_focus,
    hwemtvas=hwemtvas,
    hwdetvas=hwdetvas,
)

print(f"SEM: {sem_stream}")
print(f"FIB: {fib_stream}")

# acquisitionStreams for FLM

ccd = model.getComponent(role="ccd")
light = model.getComponent(role="light")
focus = model.getComponent(role="focus")
em_filter = model.getComponent(role="filter")
fm_stream = FluoStream(
    name="FM",
    detector=ccd,
    dataflow=ccd.data,
    emitter=light,
    em_filter=em_filter,
    focuser=focus,
    # opm=self._main_data_model.opm,
    # detvas={"exposureTime"},
)

from odemis.acq.move import MicroscopePostureManager
pm = MicroscopePostureManager(model.getMicroscope())

In [ ]:
from odemis import model
from odemis.acq.feature import CryoFeature, model, read_features, save_features, get_feature_position_at_posture
from odemis.acq.move import MicroscopePostureManager, POSITION_NAMES, SEM_IMAGING, MILLING, FM_IMAGING
from odemis.util.filename import make_unique_name
from odemis.acq.milling.tasks import load_milling_tasks, draw_milling_tasks
from odemis.acq.millmng import run_automated_milling, MillingWorkflowTask
from datetime import datetime

logging.info(f"HELLO WORLD")
# format as YYYY-MM-DD_HH-MM-SS
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
PROJECT_NAME = f"milling-project-{timestamp}"
PROJECT_PATH = os.path.join(os.getcwd(), "automated-milling-projects", PROJECT_NAME)
os.makedirs(PROJECT_PATH, exist_ok=True)

# load milling tasks
MILLING_TASKS_PATH = "/home/patrick/development/odemis/scripts/automated-milling-projects/milling_tasks.yaml"
milling_tasks = load_milling_tasks(MILLING_TASKS_PATH)

# create features
features = read_features("/home/patrick/Pictures/my-project-11")
pm = MicroscopePostureManager(model.getMicroscope())

for f in features:
    posture = pm.getCurrentPostureLabel(f.stage_position.value)
    
    fm_position = get_feature_position_at_posture(pm, f, FM_IMAGING)
    sem_position = get_feature_position_at_posture(pm, f, SEM_IMAGING)  # TODO: make sure we don't update the SEM md now when switchiing between SEM/FM it messes this up
    mill_position = get_feature_position_at_posture(pm, f, MILLING)

    # assign the milling tasks
    f.milling_tasks = milling_tasks

    # assign reference images
    fut = acquire([fib_stream])
    images = fut.result()[0]
    reference_image = images[0]

    # save the image, don't assign it
    f.path = os.path.join(PROJECT_PATH, f.name.value)
    os.makedirs(f.path, exist_ok=True)

    from odemis.dataio.tiff import export
    filename = os.path.join(f.path, f"{f.name.value}-Reference-Alignment-FIB.ome.tiff") 
    export(filename,reference_image)

    fig = draw_milling_tasks(reference_image, f.milling_tasks)
    plt.title(f.name.value)
    plt.show()

    pprint(f.posture_positions)

# save features
save_features(PROJECT_PATH, features)


In [ ]:
task_list = [MillingWorkflowTask.RoughMilling, MillingWorkflowTask.Polishing] #, MillingWorkflowTask.Polishing]

f: model.ProgressiveFuture = run_automated_milling(
                            features=features,
                            stage=stage, 
                            sem_stream=sem_stream, 
                            fib_stream=fib_stream, 
                            task_list=task_list,
                            )
f.result()

